# Model Results Visualization
This notebook is intended to pull model artifacts from the Neptune experiment logger and run it on a few samples from the dataset to see visualize how the different model's are segmenting the data. 

In [34]:
from neptune import Session
import pickle
from rasterio.plot import show
import pytorch_lightning as pl
import torch
import torchvision
import zipfile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [17]:
image_types = ['full_channel', 'rgb', 'ir', 'hsv', 'hsv_with_ir', 'veg_index']

In [18]:
f = open("image_samples.pkl", "rb")
image_samples = pickle.load(f)

In [19]:
# create session and import my project
sesh = Session.with_default_backend(api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5"
                                "lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOGE5NDI0YTktNmE2ZC00ZWZjLTlkMjAtNjNmMTIwM2Q2ZTQzIn0=")
project = sesh.get_project("maxzvyagin/GIS")

In [20]:
experiments = project.get_experiments()

In [21]:
experiments

[Experiment(GIS-132),
 Experiment(GIS-133),
 Experiment(GIS-134),
 Experiment(GIS-135)]

In [37]:
for e in experiments:
    # load in the model
    e.download_artifacts()
    with zipfile.ZipFile('output.zip') as zip_ref:
        zip_ref.extractall()
    f = open('output/latest_model.pkl', 'rb')
    model = pickle.load(f)
    f.close()
    # check the image type, and get image samples for corresponding image type
    i_type = e.get_parameters()['image_type']
    # run the model on each of the samples and show results
    # print experiment id, name, image type, and training/test loss
    print(e.get_properties(), e.get_numeric_channels_values())
    # show mask, and then show the results from each 
    for test in image_samples[i_type]:
        fig, (m_axis, i_axis) = pyplot.subplots(1, 2, figsize=(15, 15))
        res = model(test['image'])
        show(test['mask'], m_axis, title="Ground Truth")
        show(res, i_axis, title="Model Prediction")
    break

UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.